In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**CLEANSING WEATHER DATA**

In [2]:
import pandas as pd

# Load your data
data = pd.read_csv("/content/drive/MyDrive/Engg680_Digital_Engineering_Project_Group_5/Data_Collection /Weather_Data /en_climate_daily_AB_3031092_2014_P1D.csv")


columns_to_delete = ['Dir of Max Gust (10s deg)', 'Spd of Max Gust (km/h)']

# Drop the specified columns
data = data.drop(columns=columns_to_delete)


# 2. Drop columns with many missing values (if they lack significant information)
data = data.dropna(axis=1, thresh=int(0.6 * len(data)))  # Only keep columns with <60% missing data

if 'Date/Time' in data.columns:
    data['Date/Time'] = pd.to_datetime(data['Date/Time'], errors='coerce')
    data = data.dropna(subset=['Date/Time'])  # Drop rows with invalid 'Date/Time'
    data['Month'] = data['Date/Time'].dt.month  # Extract month

    data.insert(data.columns.get_loc('Date/Time') + 1, 'Day_of_Year', data['Date/Time'].dt.dayofyear)
    columns_to_fill = [
    'Max Temp (°C)', 'Min Temp (°C)', 'Mean Temp (°C)',
    'Total Rain (mm)', 'Total Snow (cm)', 'Total Precip (mm)',
    'Heat Deg Days (°C)', 'Cool Deg Days (°C)',
    'Dir of Max Gust (10s deg)', 'Spd of Max Gust (km/h)'
]
for column in columns_to_fill:
    if column in data.columns:
        data[column] = data.groupby('Month')[column].transform(lambda x: x.fillna(x.mean()))

# Drop the temporary 'Month' column after filling missing values
data = data.drop(columns=['Month'])
data.to_csv('/content/drive/MyDrive/Engg680_Digital_Engineering_Project_Group_5/Data_Cleansing/Weather_Data/Cleaned_weather_data_2014.csv', index=True)